# How to load data to BigQuery

Demonstrate a few different ways to load data to BigQuery from a Python 3 notebook.
* [bq](https://cloud.google.com/bigquery/docs/bq-command-line-tool) command line tool
* [gcloud python client](https://googleapis.github.io/google-cloud-python/latest/bigquery/usage/index.html#bigquery-basics)
* [pandas-gbq](https://pandas-gbq.readthedocs.io/en/latest/)

For files, the `bq` tool or the gcloud-python-client are great and work the same. Choose whichever one you like more.

For dataframes in memory, the pandas-gbq client is a great way to go (no need to write it out to a file first).

# Setup

First, be sure to run notebook **`Python environment setup`** in this workspace.

Then in this section we:

1. load the needed python packages
2. set some constants to be used as parameters to our BigQuery load jobs

In [1]:
from google.cloud import bigquery
from google.cloud.bigquery import LoadJobConfig
from google.cloud.bigquery import SchemaField

import os
import numpy as np
import pandas as pd
import time

Edit these global variables in your clone of this notebook if you do not have permission to WRITE data to this native Google Cloud Platform project.
* The destination BigQuery dataset should already exist. Your pet account must have WRITE access to it.
* The remaining cells can be run as-is.

In [2]:
# CHANGE THESE VARIABLES, IF NEEDED
DESTINATION_PROJECT_ID = 'terra-resources'
DESTINATION_DATASET = 'autodelete_after_one_day'

In [3]:
BILLING_PROJECT_ID = os.environ['GOOGLE_PROJECT']
CSV_PATH = 'gs://genomics-public-data/platinum-genomes/other/platinum_genomes_sample_info.csv'
# Also try this CSV which will yield some autodetect errors.
# gs://genomics-public-data/1000-genomes/other/sample_info/sample_info.csv

# Load data to BigQuery from a CSV

## Via the `bq` command line tool

In [4]:
DESTINATION_TABLE = 'py3_bq_' + time.strftime("%Y%m%d_%H%M%S")

In [5]:
%%bash -s "$DESTINATION_PROJECT_ID" "$DESTINATION_DATASET" "$DESTINATION_TABLE" "$CSV_PATH"

bq --project ${1} load --autodetect ${2}.${3} ${4}

Waiting on bqjob_r14f2c5ffe6ba3b9d_0000016a7058860c_1 ... (1s) Current status: DONE   

Show the table schema.

In [6]:
%%bash -s "$DESTINATION_PROJECT_ID" "$DESTINATION_DATASET" "$DESTINATION_TABLE"

bq --project ${1} show ${2}.${3}

Table terra-resources:autodelete_after_one_day.py3_bq_20190430_222329

   Last modified                  Schema                 Total Rows   Total Bytes     Expiration      Time Partitioning   Labels  
 ----------------- ------------------------------------ ------------ ------------- ----------------- ------------------- -------- 
  30 Apr 22:23:32   |- Catalog_ID: string                17           1441          01 May 22:23:32                               
                    |- Description: string                                                                                        
                    |- Gender: string                                                                                             
                    |- Race: string                                                                                               
                    |- Family_Member: integer                                                                                     
            

## Via the gcloud Python client

https://googlecloudplatform.github.io/google-cloud-python/latest/bigquery/generated/google.cloud.bigquery.job.LoadJobConfig.html

In [7]:
client = bigquery.Client(project=os.environ['GOOGLE_PROJECT'])
DESTINATION_TABLE = 'py3_gcloud_py_client_' + time.strftime("%Y%m%d_%H%M%S")

In [8]:
table_ref = client.dataset(DESTINATION_DATASET,
                           project=DESTINATION_PROJECT_ID).table(DESTINATION_TABLE)

# https://googlecloudplatform.github.io/google-cloud-python/latest/bigquery/generated/google.cloud.bigquery.job.LoadJobConfig.html
job_config = LoadJobConfig()
job_config.source_format = bigquery.SourceFormat.CSV
job_config.autodetect = True

load_job = client.load_table_from_uri(CSV_PATH, table_ref, job_config=job_config)
print('Loading {}, starting job {}'.format(DESTINATION_TABLE, load_job.job_id))

Loading py3_gcloud_py_client_20190430_222336, starting job 221b0cec-3079-430f-8ca6-c9a813e100f4


In [9]:
# Waits for table load to complete.
load_job.result()
print('Job finished.')

Job finished.


In [10]:
load_job.errors

In [11]:
table = client.get_table(table_ref)  # API Request
print(table.schema)

[SchemaField('Catalog_ID', 'STRING', 'NULLABLE', None, ()), SchemaField('Description', 'STRING', 'NULLABLE', None, ()), SchemaField('Gender', 'STRING', 'NULLABLE', None, ()), SchemaField('Race', 'STRING', 'NULLABLE', None, ()), SchemaField('Family_Member', 'INTEGER', 'NULLABLE', None, ()), SchemaField('Relationship_to_Proband', 'STRING', 'NULLABLE', None, ()), SchemaField('CT_Desc', 'STRING', 'NULLABLE', None, ()), SchemaField('Cum_Pdl', 'STRING', 'NULLABLE', None, ())]


In [12]:
print(table.description)

None


In [13]:
print(table.num_rows)

17


# Load data to BigQuery from a dataframe

## Via pandas-gbq

In [14]:
DESTINATION_TABLE = 'py3_pandas_gbq_' + time.strftime("%Y%m%d_%H%M%S")

In [15]:
df = pd.DataFrame(np.random.randint(low=0, high=10, size=(5, 5)),
                  columns=['a', 'b', 'c', 'd', 'e'])

In [16]:
df.to_gbq(destination_table='.'.join([DESTINATION_DATASET, DESTINATION_TABLE]),
          project_id=DESTINATION_PROJECT_ID)

In [17]:
pd.io.gbq.read_gbq('SELECT COUNT(*) AS cnt FROM `{}.{}.{}`'.format(DESTINATION_PROJECT_ID,
                                                                   DESTINATION_DATASET,
                                                                   DESTINATION_TABLE),
                   project_id=BILLING_PROJECT_ID,
                   dialect='standard')

,cnt
0,5


# Provenance

In [18]:
import datetime
print(datetime.datetime.now())

2019-04-30 22:24:14.943891


In [19]:
!pip3 freeze

absl-py==0.7.1
astor==0.7.1
descartes==1.1.0
firecloud==0.16.22
gast==0.2.2
google-api-core==1.10.0
google-auth-oauthlib==0.3.0
google-cloud-bigquery==1.11.2
google-cloud-core==0.29.1
h5py==2.9.0
ibis-framework==1.0.0
Keras-Applications==1.0.7
Keras-Preprocessing==1.0.9
Markdown==3.1
mizani==0.5.4
mock==2.0.0
multipledispatch==0.6.0
oauthlib==3.0.1
palettable==3.1.1
pandas==0.24.2
pandas-gbq==0.10.0
pbr==5.2.0
plotnine==0.5.1
pydata-google-auth==0.1.3
regex==2019.4.14
requests-oauthlib==1.2.0
tensorboard==1.13.1
tensorflow==1.13.1
tensorflow-estimator==1.13.0
termcolor==1.1.0
toolz==0.9.0
Werkzeug==0.15.2


Copyright 2018 The Broad Institute, Inc., Verily Life Sciences, LLC All rights reserved.

This software may be modified and distributed under the terms of the BSD license. See the LICENSE file for details.